# 🖼️ CIFAR-10 图像分类任务 (Task 2)

## 📋 项目概述
本项目实现了基于深度学习的 CIFAR-10 图像分类任务，使用卷积神经网络对 10 类彩色图像进行分类识别。

## 🎯 任务目标
- **数据集**：CIFAR-10（60,000 张 32×32 彩色图像）
- **类别数**：10 类（飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车）
- **模型架构**：基础 CNN（卷积神经网络 + 全连接层）
- **目标准确率**：67% ~ 69%


## 📊 训练配置
| 参数 | 值 |
|------|-----|
| Batch Size | 64 |
| Epochs | 10 |
| Learning Rate | 0.01 |
| Optimizer | SGD (momentum=0.9) |
| Loss Function | CrossEntropyLoss |

---

In [ ]:
#导入模块与超参数设置
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 训练超参数
batch_size = 64
epochs = 10
learning_rate = 0.01
momentum = 0.9
log_interval = 100  # 每 100 个 batch 输出一次损失

# 数据归一化参数
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std = (0.2023, 0.1994, 0.2010)

In [ ]:
#数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std)
])

train_dataset = datasets.CIFAR10(root='./DS/CIFAR10',
                                 train=True,
                                 download=True,
                                 transform=transform)

train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.CIFAR10(root='./DS/CIFAR10',
                                 train=False,
                                 download=True,
                                 transform=transform)
                
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

In [ ]:
#定义模型

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CIFAR10 输入: (batch, 3, 32, 32)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)  # 展平
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
#定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
#训练函数
def train(epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % log_interval == log_interval - 1:
            print(f'[Epoch {epoch + 1}, Batch {batch_idx + 1}] loss: {running_loss / log_interval:.3f}')
            running_loss = 0.0

In [ ]:
#测试函数
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total

    print(f'Accuracy on test set: {accuracy:.2f}%')   

In [ ]:
#开始训练
for epoch in range(epochs):
    train(epoch)
    test()